# SW중심대학 디지털 경진대회_SW와 생성AI의 만남 : AI부문
 - 이 AI 경진대회에서는 5초 분량의 오디오 샘플에서 진짜 사람 목소리와 AI가 생성한 가짜 목소리를 정확하게 구분할 수 있는 모델을 개발하는 것이 목표입니다.
 - 이 작업은 보안, 사기 감지 및 오디오 처리 기술 향상 등 다양한 분야에서 매우 중요합니다.

## Imports
모델 학습 및 추론에 사용할 라이브러리들을 불러옵니다.

In [ ]:
import os
import random

import torch
import torchaudio

import numpy as np
import pandas as pd

from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm.notebook import tqdm

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Set CUDA Device Number 0~7
DEVICE_NUM = 1

device = torch.device("cpu")
if torch.cuda.is_available():
    torch.cuda.set_device(DEVICE_NUM)
    device = torch.device("cuda")
print("INFO: Using device -", device)

In [ ]:
!nvidia-smi

## Config
- 딥러닝 모델을 학습하기 전에 설정해야하는 다양한 매개변수를 정의하는 설정 클래스입니다.
- 클래스를 사용하여 학습에 필요한 설정 값을 미리 지정합니다.

##### 오디오 신호
- 우리가 듣는 소리는 공기의 압력 변화로, 이것을 디지털 신호로 변환한 것이 오디오 신호입니다.
- 이 신호는 시간에 따라 변하는 진폭 값을 가지고 있습니다.

In [ ]:
class CONFIG:
    """ Configuration Class """
    SEED = 202102545  # 재현성을 위해 랜덤 시드 고정
    
    """ SR(Sample Rate)
    - 오디오 데이터의 샘플링 레이트를 설정합니다.
    - 높은 샘플링 레이트는 더 높은 주파수의 소리를 캡처할 수 있지만, 처리에 더 많은 계산 자원이 필요합니다.
    - 오디오 데이터의 초당 샘플 수를 정의합니다.
    """
    SR = 32000

    """ ROOT_FOLDER
    - 데이터셋의 루트 폴더 경로를 설정합니다.
    """
    ROOT_FOLDER = os.path.join(".", "data")
    
    """ BATCH_SIZE
    - 학습 시 한 번에 처리할 데이터 샘플의 수를 정의합니다
    - 큰 배치 크기는 메모리 사용량을 증가시키지만, 학습 속도를 높입니다.
    """
    BATCH_SIZE = 100
    
    """ N_EPOCHS
    - 전체 데이터셋을 학습할 횟수를 정의합니다.
    - 에폭 수가 너무 적으면 과소적합이 발생할 수 있고, 너무 많으면 과적합이 발생할 수 있습니다.
    """
    N_EPOCHS = 200
    
    """ LR (Learning Rate)
    - 모델의 가중치를 업데이트할 때 사용되는 학습 속도를 정의합니다.
    - 학습률이 너무 크면 학습이 불안정해질 수 있고, 너무 작으면 학습 속도가 느려집니다.
    """
    LR = 1e-5

In [ ]:
def seed_everything(seed):
    """ Fixed RandomSeed
    아래의 코드는 머신러닝이나 딥러닝 모델을 훈련할 때, 결과의 재현성을 보장하기 위해 사용되는 함수입니다.
    이 함수는 다양한 랜덤 시드를 고정하여, 실행할 때마다 동일한 결과를 얻기 위해 사용됩니다.
    """
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CONFIG.SEED)  # Seed 고정

## Dataset

In [ ]:
from torchvision.datasets import utils
from sklearn.model_selection import train_test_split as split

utils.tqdm = tqdm


class ContrastingVoiceDataset(Dataset):
    download_url = "https://drive.usercontent.google.com/download?id=1hi1dibkHyFbaxAteLlZJw6r3g9ddd4Lf&export=download&authuser=0&confirm=t&uuid=c40c278b-d74b-4b75-bc79-09e8a3ccffa4&at=APZUnTUvIVFVM9gjGNUCmDb4YZCy%3A1719807236671"
    
    @classmethod
    def download(cls, root='./data', filename="download.zip", md5=None):
        cls.download_root = root
        filepath = os.path.join(root, filename)
        if not os.path.exists(filepath):
            utils.download_and_extract_archive(cls.download_url, root, root, filename, md5)
            print("Extraction completed.")
        else:
            print(f"File already exists in {filepath}")
    
    @classmethod
    def get_dataset_path(cls, root, train=True):
        return os.path.join(root, "train.csv" if train else "test.csv")

    @property
    def submission_form_path(cls):
        return os.path.join(cls.download_root, "sample_submission.csv")
    
    def __init__(self, root="./data", train=True, split_ratio=1, transform=None):
        """
        Voice Dataset for Contrastive Learning
        
        :param root: The path to the data directory
        :param train: is train or test
        :param split_ratio: split ratio for train(can be 0.5 or above) and valid(can be lower than 0.5) set
        :param transform: data transformer
        :param target_transform: label transformer
        """
        super().__init__()
        self.download(root)
        self.download_root = root
        self.is_train = train

        raw_data = self._load_data(self.get_dataset_path(root, train), split_ratio if split_ratio >= 0.5 else 1-split_ratio)
        if split_ratio >= 0.5:
            self.raw_data, _ = raw_data
        else:
            _, self.raw_data = raw_data
            
        self.multi_label = 'path' not in self.raw_data.columns
        
        self.ids = self.raw_data['id']
        if self.multi_label:
            self.data = [self.raw_data['path0'], self.raw_data['path1']]
            self.label = [self.raw_data['label0'], self.raw_data['label1']]
        else:
            self.data = self.raw_data['path']
            if 'label' in self.raw_data.columns:
                self.label = self.raw_data['label']
            else:
                self.label = None

        self.transforms(transform)
    
    @staticmethod
    def _load_data(dataset_path, split_ratio):
        df = pd.read_csv(dataset_path)
        
        if split_ratio == 1 or split_ratio == 0:
            return (df, None) if split_ratio == 1 else (None, df)
            
        df1, df2, _, _ = split(df, df['label'], test_size=1-split_ratio, random_state=202102545)
        return df1, df2
    
    @staticmethod
    def get_id_from_path(path: str):
        return [pth for pth in path.replace("/0.ogg", ".ogg").split("/") if '.ogg' in pth][0].replace(".ogg", "")

    def transforms(self, transform=None):
        if transform is not None:
            if not isinstance(transform, list) and not isinstance(transform, tuple):
                transform = [transform]
            for t in transform:
                self.ids, self.data, self.label = t(self.ids, self.data, self.label)

    def __len__(self):
        if self.multi_label:
            return len(self.data[0])
        return len(self.data)

    def __getitem__(self, index):
        if self.label is not None:
            if self.multi_label:
                return [d[index] for d in self.data], [l[index] for l in self.label], self.ids[index]
            return self.data[index], self.label[index], self.ids[index]
        return self.data[index], self.ids[index]

In [ ]:
def to_list(multi_label=False):
    def to_list_inner(ids, datas, labels):
        ids = [_id for _id in ids]
        if not multi_label:
            datas, labels = [datas], [labels]
        datas = [[d for d in data] for data in datas]
        try:
            labels = [[l for l in label] for label in labels]
        except TypeError:
            pass
        if not multi_label:
            datas, labels = datas[0], labels[0]
        return ids, datas, labels
    return to_list_inner

In [ ]:
train_dataset = ContrastingVoiceDataset(root=CONFIG.ROOT_FOLDER, train=True, split_ratio=0.8, transform=to_list())
valid_dataset = ContrastingVoiceDataset(root=CONFIG.ROOT_FOLDER, train=True, split_ratio=0.2, transform=to_list())
test_dataset = ContrastingVoiceDataset(root=CONFIG.ROOT_FOLDER, train=False, split_ratio=1, transform=to_list())

print("Query Dataset for checking:", train_dataset[0])
train_dataset.raw_data

#### Data Transformation

In [ ]:
from huggingface_hub import hf_hub_download
import wespeaker


def get_resnet152():
    model_id = "Wespeaker/wespeaker-voxceleb-resnet152-LM"
    model_name = model_id.replace("Wespeaker/wespeaker-", "").replace("-", "_")

    root_dir = hf_hub_download(model_id, filename=model_name+".onnx").replace(model_name+".onnx", "")

    import os
    if not os.path.isfile(root_dir+"avg_model.pt"):
        os.rename(hf_hub_download(model_id, filename=model_name+".pt"), root_dir+"avg_model.pt")
    if not os.path.isfile(root_dir+"config.yaml"):
        os.rename(hf_hub_download(model_id, filename=model_name+".yaml"), root_dir+"config.yaml")

    resnet = wespeaker.load_model_local(root_dir)
    resnet.set_gpu(-1 if device == torch.device('cpu') else 0)

    def resnet152(pcm, sample_rate=None):
        if isinstance(pcm, str):
            return resnet.extract_embedding(pcm)
        else:
            pass  # TODO: 메모리에 로드된 상태의 오디오 처리 코드 필요
            #return extract_embedding(resnet, pcm, sample_rate)

    print(f"ResNet152 Model Loaded on {resnet.device}")
    return resnet152

In [ ]:
train_embedding_file = "train_embedding.pt"
valid_embedding_file = "valid_embedding.pt"


def get_pretrained_embedding():
    if not os.path.isfile(train_embedding_file) \
            or not os.path.isfile(valid_embedding_file):
        return get_resnet152()
    else:
        train_embedding = torch.load(train_embedding_file)
        valid_embedding = torch.load(valid_embedding_file)
        dataset_list = {
            len(train_embedding): train_embedding,
            len(valid_embedding): valid_embedding,
        }
        print("INFO: Pretrained Voice Embedding loaded.", dataset_list.keys())
        
        def load_embedding(dataset):
            return dataset_list[len(dataset)]
        
        load_embedding.__dict__['pretrained'] = True
        return load_embedding

In [ ]:
def to_embedding(pretrained=get_pretrained_embedding(), sample_rate=CONFIG.SR):
    get_pth = lambda path: os.path.join(CONFIG.ROOT_FOLDER, *path[1:].split("/"))
    
    if not pretrained:
        def extract_embedding(ids, datas, labels):  # TODO: 임베딩 코드 추가 필요
            return ids, [torchaudio.load(data) for data in datas], labels
        return extract_embedding
    
    def pretrained_embedding(ids, dataset, labels):
        if pretrained.__dict__.get('pretrained'):
            new_dataset = pretrained(dataset)
            print("INFO: Voice Embedding extracted.")
        else:
            new_dataset = []

            for data in tqdm(dataset):
                new_dataset.append(pretrained(get_pth(data), sample_rate))

            dataset_size = len(new_dataset)

            torch.save(new_dataset, "nonamed.pt")
            if dataset_size == len(train_dataset.raw_data):
                os.rename("nonamed.pt", "train_embedding.pt")
            elif dataset_size == len(valid_dataset.raw_data):
                os.rename("nonamed.pt", "valid_embedding.pt")
            else:
                raise ValueError(f"Invalid Dataset Size - Could not find relevant dataset sized {dataset_size}.")
                
            print("INFO: Voice Embedding saved.")
                
        return ids, new_dataset, labels  # [pretrained(get_pth(path), sample_rate) for path in dataset]
    
    return pretrained_embedding

In [ ]:
def to_tensor_label(ids, datas, labels):
    if labels:
        labels = [torch.tensor([1, 0]) if lb == "fake" else torch.tensor([0, 1]) for lb in labels]  # [fake, real]
    return ids, datas, labels

In [ ]:
train_dataset.transforms(transform=[to_embedding(), to_tensor_label])
valid_dataset.transforms(transform=[to_embedding(), to_tensor_label])

In [ ]:
for dataset, index in zip(train_dataset, range(5)):
    print(f"Dataset {index}: {'FAKE' if dataset[1] == torch.tensor([0]) else 'REAL'}", dataset[0])

In [ ]:
for dataset, index in zip(valid_dataset, range(5)):
    print(f"Dataset {index}: {'FAKE' if dataset[1] == torch.tensor([0]) else 'REAL'}", dataset[0])

In [ ]:
for dataset, index in zip(test_dataset, range(1)):
    print(f"Dataset {index}:", dataset[0])

## DataLoader
    - DataLoader는 구축된 데이터셋에서 배치크기(batch_size)에 맞게 데이터를 추출하고, 필요에 따라 섞거나(shuffle=True) 순서대로 반환(shuffle=False)하는 역할을 합니다.
    - 훈련 데이터(train_loader)는 일반적으로 섞어서 모델이 데이터에 덜 편향되게 학습하도록하며,
      검증 데이터(val_loader)는 모델 성능 평가를 위해 순서대로 사용하고,
      테스트 데이터(test_loader)는 최종적인 추론을 위해 사용합니다.

    이렇게 DataLoader를 사용함으로써, 효율적인 데이터 처리와 모델 학습 및 평가가 가능해집니다.

In [ ]:
BATCH_SIZE = CONFIG.BATCH_SIZE

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

## Define Model

In [ ]:
class VoiceEncoder(nn.Module):
    """ Voice Encoder Model """
    
    def __init__(self, embedding_dim, hidden_dim, output_size):
        super().__init__()
        self.fc1 = nn.Linear(embedding_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        h1 = self.relu(self.fc1(x))
        out = self.fc2(h1)
        return out

In [ ]:
class VoiceDiscriminator(nn.Module):
    """ Voice Discriminator Model """
    
    def __init__(self, embedding_dim, hidden_size, latent_size):
        super().__init__()
        self.encoder = VoiceEncoder(embedding_dim, hidden_size, latent_size)
        self.classifier = nn.Linear(latent_size, 2)

    def forward(self, x):
        encoded = self.encoder(x)
        out = self.classifier(encoded)
        return F.sigmoid(out)

In [ ]:
# 모델 파라미터 지정
model_params = dict(
    embedding_dim=len(train_dataset[0][0]),
    hidden_size=512,
    latent_size=128
)
model_params

In [ ]:
# 모델 생성
voice_discrimination_model = VoiceDiscriminator(**model_params)
discriminator = voice_discrimination_model
discriminator.to(device)

In [ ]:
# BinaryCrossEntropy
criterion = nn.BCELoss().to(device)

In [ ]:
# Adam optimizer
optimizer = torch.optim.Adam(params=discriminator.parameters(), lr=CONFIG.LR)

## Train & Validation

In [ ]:
def multiLabel_AUC(y_true, y_scores):
    auc_scores = []
    for i in range(y_true.shape[1]):
        auc = roc_auc_score(y_true[:, i], y_scores[:, i])
        auc_scores.append(auc)
    mean_auc_score = np.mean(auc_scores)
    return mean_auc_score

In [ ]:
def validation(model, criterion, val_loader, device):
    """
    1. 모델 평가 모드 설정
        - model.eval()을 호출하여 모델을 평가 모드로 설정합니다.
    2. 손실 계산 및 예측 수집
        - val_loader에서 배치별로 스펙트로그램과 레이블을 불러와, 모델을 통해 예측을 수행합니다.
        - 손실 값을 기록하고 예측된 확률과 실제 레이블을 수집합니다.
    3. 성능 평가
        - 수집된 모든 예측과 실제 레이블을 사용해 multiLabel_AUC 함수를 호출하여 평균 AUC 점수를 계산합니다.
    """
    
    model.eval()
    val_loss, all_labels, all_probs = [], [], []

    with torch.no_grad():
        for features, labels, _ in val_loader:
            features = features.float().to(device)
            labels = labels.float().to(device)

            probs = model(features)

            loss = criterion(probs, labels)

            val_loss.append(loss.item())

            all_labels.append(labels.cpu().numpy())
            all_probs.append(probs.cpu().numpy())

        _val_loss = np.mean(val_loss)

        all_labels = np.concatenate(all_labels, axis=0)
        all_probs = np.concatenate(all_probs, axis=0)

        # Calculate AUC score
        auc_score = multiLabel_AUC(all_labels, all_probs)

    return _val_loss, auc_score

In [ ]:
from sklearn.metrics import roc_auc_score

def train(model, optimizer, train_loader, val_loader, device):
    """
    1. 모델 및 Loss 함수 설정
        - 모델을 주어진 device(GPU 또는 CPU)로 이동하고, BCELoss(Binary Cross Entropy)를 Loss 함수로 설정합니다.
    2. 최적의 모델 저장을 위한 변수 초기화
        - 학습 과정동안 가장 좋은 검증 점수를 가진 모델을 추적하기 위해 최적의 점수(best_val_score)와 모델(best_model)을 저장할 변수를 초기화합니다.
    3.에포크별 훈련
        - 설정된 에포크 수(CONFIG.N_EPOCHS)만큼 반복하여 모델을 훈련합니다.
        - train_loader에서 배치별로 스펙트로그램과 레이블을 불러옵니다.
        - 모델을 통해 예측을 수행하고 , 손실을 계산합니다.
        - loss.backward()를 수행하고, 파라미터를 업데이트 합니다.
        - 이후, 훈련 손실을 기록합니다.
    4. 검증 및 최적 모델 업데이트
        - 각 에포크 후에 검증 함수를 호출해 모델의 성능을 평가하며, 현재 에포크에서 더 나은 검증 점수를 달성하면 이 모델을 최적의 모델로 간주하고 저장합니다.
    """
    
    model.to(device)
    criterion = nn.BCELoss().to(device)
    
    best_val_score = 0
    best_model = None
    
    train_length = len(train_loader)
    valid_length = len(val_loader)
    
    epochs = tqdm(range(1, CONFIG.N_EPOCHS+1), desc="Running Epochs")
    train_loader = tqdm(train_loader, desc="Training")
    valid_loader = tqdm(valid_loader, desc="Validation")
    
    for epoch in epochs:
        train_loader.reset(total=train_length)
        valid_loader.reset(total=valid_length)
        
        model.train()
        train_loss = []
        for features, labels, _ in train_loader:
            features = features.float().to(device)
            labels = labels.float().to(device)

            optimizer.zero_grad()

            output = model(features)
            loss = criterion(output, labels)

            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f"Epoch [{epoch}], Train Loss : [{_train_loss:.6f}] Val Loss : [{_val_loss:.6f}] Val AUC : [{_val_score:.6%}]")

        if best_val_score < _val_score:
            best_val_score = _val_score
            best_model = model
    
    return best_model, best_val_score

In [ ]:
infer_model, ifer_score = train(discriminator, optimizer, train_loader, valid_loader, device)

In [ ]:
if not os.path.isdir(os.path.join(".", "models")):
    os.mkdir(os.path.join(".", "models"))

# Model Save
save_path = os.path.join(".", "models", f"baseline_wespeaker_model_acc_{ifer_score:6%}.pt")
torch.save(discriminator.state_dict(), save_path)
print(f"Model saved to {save_path}")

### Inference
테스트 데이터셋에 대한 추론은 다음 순서로 진행됩니다.

1. 모델 및 디바이스 설정
    - 모델을 주어진 device(GPU 또는 CPU)로 이동시키고, 평가모드로 전환합니다.
2. 예측 수행
    - 예측 결과를 저장한 빈 리스트를 초기화하고 test_loader에서 배치별로 데이터를 불러와 예측을 수행합니다.
    - 각 배치에 대해 스펙트로그램 데이터를 device로 이동시킵니다.
    - 모델 예측 확률(probs)을 계산합니다.
    - 예측 확률을 predictions리스트에 추가합니다.

In [ ]:
def inference(model, test_loader, device, to_embedding=get_resnet152()):
    get_pth = lambda path: os.path.join(CONFIG.ROOT_FOLDER, *path[1:].split("/"))
    
    predictions = []
    
    model.to(device)
    model.eval()
    with torch.no_grad():
        for features, _ in tqdm(test_loader):
            path = get_pth(features[0])
            print(path)
            features = to_embedding(path, CONFIG.SR).to(device)
            
            probs = model(features)

            probs  = probs.cpu().detach().numpy()
            predictions += probs.tolist()
    return predictions

In [ ]:
predicted_labels = inference(discriminator, test_loader, device)

### Submission
추론 결과를 제출 양식에 덮어 씌워 CSV 파일로 생성하는 과정은 다음과 같습니다.

1. 제출 양식 로드
    - pd.read_csv('./sample_submission.csv')를 사용하여 제출을 위한 샘플 형식 파일을 로드합니다.
    - 이 파일은 일반적으로 각 테스트 샘플에 대한 ID와 예측해야 하는 필드가 포함된 템플릿 형태를 가지고 있습니다.
2. 예측 결과 할당
    - submit.iloc[:,1:] = preds 추론함수(inference)에서 반환된 예측결과(preds)를 샘플 제출 파일에 2번째 열부터 할당합니다.
3. 제출 파일 저장
    - 수정된 제출 파일을 baseline_submit 이란 이름의 CSV 파일로 저장합니다.
    - index=False는 파일 저장시 추가적인 index가 발생하지 않도록 설정하여, 제작한 제출 파일과 동일한 형태의 파일을 저장합니다.

In [ ]:
submit = pd.read_csv(test_dataset.submission_form_path)
submit.iloc[:, 1:] = torch.tensor(predicted_labels).cpu().numpy()
submit.head()

In [ ]:
submit.to_csv(f"{model_id}_submit.csv", index=False)